# Ce code marche avec doc_sumary_only.py
# et la base de données situées dans PDF_CGV
## a lire!

In [1]:
# ce script permet de lancer le script doc_summary_only.py pour générer les sommaires de tout les documents
import subprocess
subprocess.run(['python', 'doc_summary_only.py'])

CompletedProcess(args=['python', 'doc_summary_only.py'], returncode=1)

In [2]:
# !pip install PyMuPDF
# !pip install pandas
import fitz  # PyMuPDF
import os
import pandas as pd


In [3]:

def one_file_summary(f):
    doc = fitz.open(f)
    page_sommaire=[]
    page_litige=[]
    # phase 1 : rechercher les pages du sommaire et du litige
    for num_page in range(len(doc)):
        page = doc[num_page]
        if page.search_for("sommaire") != []:
            page_sommaire.append(num_page)
        if page.search_for("litige") != []:
            page_litige.append(num_page)
        else:
            continue
            
    #  phase 2 enregistrer le sommaire dans un nouveau fichier
    
    try:
        new_pdf = fitz.open()
        new_pdf.insert_pdf(doc, from_page=page_sommaire[0], to_page=page_litige[0])
        f_sum = os.path.join("sommaire", "sum_" + f)
        new_pdf.save( f_sum  )
        #print("page_sommaire", page_sommaire[0])   
        #print("page_litige", page_litige[0])
        doc.close()

    except:
        print("Erreur", f)
    # pour le cas de la picardie, bug du a l'absence du mot sommaire dans le document on peut le faire manuellement
    return new_pdf


## Litteralement recupere le coeur du document. extraie  le texte, size_font,  bbox, index et nom du fichier

In [4]:
def read_a_page(doc, page_nb, reverse =False):
    # Sélectionner une page (par exemple, la première page)
    page = doc[page_nb]

    # Extraire le texte avec l'option "dict"
    text_dict = page.get_text("dict")
    # Afficher la structure générale du dictionnaire

    # Parcourir chaque bloc de texte
    font_sizes = set()

    block_inst= []
    for block in text_dict["blocks"]:
        
        if block["type"] == 0:  # Type 0 indique un bloc de texte
            block_text = ""
            bbox = block["bbox"]

            # Arrondir les coordonnées des boîtes englobantes pour gagner de la place sur calc
            def round_bbox(bbox, decimals=1):
                return [round(value, decimals) for value in bbox]
            
            bbox = round_bbox(bbox, 2)
            numberofelement=0

            # Extraire le texte et les tailles de police de chaque span    
            if bbox[3] < 800 and bbox[1] > 15:  # Filtre les blocs de texte qui sont hors marge

                for line in block["lines"]:
                        for span in line["spans"]:
                            numberofelement += 1
                            block_text += (span["text"] + "\n")

                        # font_sizes.add(span["size"]) deprecated
                        size = span["size"]
            
                block_inst.append({"bbox": bbox, "text": block_text, "number" : numberofelement, "font_size": size})

    sorted_blocks = sorted(block_inst, key=lambda b: (b["bbox"][1], b["bbox"][0])) # trier les blocs par ordre de lecture
    return sorted_blocks

##  Fonction de création du dataset sur un fichier

In [7]:
file = "Tarifs BForBank2024 V8_WEB.pdf"
pdf = one_file_summary("../PDF_CGV/" + file)
# Pour un usage avec DNN , il faut créer un dataset de la forme suivante (text, label) 
# label = 1 pour les titres
# label = 2 pour les sous-titres
# label = 0 pour le reste

def create_dataset(pdf, file, file_number):
    index = 0
    blocks = read_a_page(pdf, 0) # a modifier pour prendre en compte les sommaires a 2 pages
    u_train = pd.DataFrame(columns=["index", "text", "label","font_size","Bbox", "file_number", "file"])

    # mettre un split "\n" pour les titr et sous-titres
    for i in range(len(blocks)):
        splitted_blocks = blocks[i]["text"].split("\n")

        for block in splitted_blocks:
            if len(block) > 6: # peux important filtre pour les petits blocs
                block = block.lower()
                if block.find("sommaire") == -1: # enlever le sommaire peux faciliter la comprehension du document
                    u_train.loc[len(u_train)] = [index, block, None, blocks[i]["font_size"], blocks[i]["bbox"],file_number, file[:-4]]
                    index +=1
    return u_train

file_0 = "Tarifs BForBank2024 V8_WEB.pdf"
pdf_0 = one_file_summary("../PDF_CGV/" + file_0)
pd_0 = create_dataset(pdf_0, file_0, 0)


# Supprimer les lignes vides
# u_train = u_train.dropna(how='all')  # 'how="all"' supprime les lignes où toutes les valeurs sont manquantes inutile ici
print(pd_0)

# on sauvegarde le fichier pandas
file_path = 'mon_dataframe2.csv'

# Sauvegarder la description en haut du fichier, puis ajouter le DataFrame
with open(file_path, 'w', encoding='utf-8') as f:
    # Ecrire la description
    f.write(file_0  + "\n")
    
    # Sauvegarder le DataFrame en dessous
    pd_0.to_csv(f, index=False)

Erreur ../PDF_CGV/Tarifs BForBank2024 V8_WEB.pdf
Erreur ../PDF_CGV/Tarifs BForBank2024 V8_WEB.pdf
    index                                               text label  font_size  \
0       0   ouverture, fonctionnement et suivi de votre c...  None       15.0   
1       1                 ouverture, transformation, clôture  None        8.0   
2       2                                  relevés de compte  None        8.0   
3       3                                    tenue de compte  None        8.0   
4       4                               demande de documents  None        8.0   
5       5                                 banque à distanc  None       15.0   
6       6                                      espace client  None        8.0   
7       7                            service client bforbank  None        8.0   
8       8              vos moyens et opérations de paiemen  None       15.0   
9       9                                          virements  None        8.0   
10     10      

# Géneraliser le code precedent a tout les fichiers pdf

In [12]:
def many_summary():
    # assign directory
    directory = '../sommaires' # directory containing the PDF files
    df = pd.DataFrame(columns=["text","index", "font_size","Bbox","file_number", "file", "label"])
    df['index'] = df['index'].astype(int)       
    df['font_size'] = df['font_size'].astype(int)
    file_number = 0 # this file number helps to keep track of the file with a number, better for trainaing               
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f) # print the file name
            pdf = one_file_summary(f)
            u_train = create_dataset(pdf, filename,file_number)
            df = pd.concat([df, u_train], ignore_index=True)    
            file_number += 1
    
        # on sauvegarde le fichier pandas qu'on va utiliser POUR APPRENTISSAGE!
        file_path = 'dataset-int.csv'

        # Sauvegarder la description en haut du fichier, puis ajouter le DataFrame
        with open(file_path, 'w', encoding='utf-8') as f:

            # Supprimer les lignes entièrement vides
            df_clean = df.dropna(how='all')
            df_clean = df_clean.loc[~((df_clean == '') | (df_clean == ',')).all(axis=1)]

            # Enregistrer le DataFrame nettoyé dans un nouveau fichier CSV
            df_clean.to_csv('dataset-print2.csv', index=False)

    return df  
    # print(df.head())
sommaire = many_summary()

../sommaires\sum_Tarifs BForBank2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs BForBank2024 V8_WEB.pdf
../sommaires\sum_Tarifs BNP2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs BNP2024 V8_WEB.pdf
../sommaires\sum_Tarifs BoursoBank2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs BoursoBank2024 V8_WEB.pdf
../sommaires\sum_Tarifs BP_Occitane2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs BP_Occitane2024 V8_WEB.pdf
../sommaires\sum_Tarifs CA_Acquitaine2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs CA_Acquitaine2024 V8_WEB.pdf
../sommaires\sum_Tarifs CA_AlpesProvence2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs CA_AlpesProvence2024 V8_WEB.pdf
../sommaires\sum_Tarifs CA_AlsacesVosqges2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs CA_AlsacesVosqges2024 V8_WEB.pdf
../sommaires\sum_Tarifs CA_AnjouMaine2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs CA_AnjouMaine2024 V8_WEB.pdf
../sommaires\sum_Tarifs CA_AtlantiqueVendee2024 V8_WEB.pdf
Erreur ../sommaires\sum_Tarifs CA_AtlantiqueVendee2024 V8_WEB.p

In [15]:
# apercus des 10 premiers lignes
sommaire.head(50)

,text,index,font_size,Bbox,file_number,file,label
0,"ouverture, fonctionnement et suivi de votre c...",0,15.000000,"[31.64, 92.02, 562.91, 111.39]",0,sum_Tarifs BForBank2024 V8_WEB,None
1,"ouverture, transformation, clôture",1,8.000000,"[31.64, 112.45, 166.92, 152.45]",0,sum_Tarifs BForBank2024 V8_WEB,None
2,relevés de compte,2,8.000000,"[31.64, 112.45, 166.92, 152.45]",0,sum_Tarifs BForBank2024 V8_WEB,None
3,tenue de compte,3,8.000000,"[31.64, 112.45, 166.92, 152.45]",0,sum_Tarifs BForBank2024 V8_WEB,None
4,demande de documents,4,8.000000,"[31.64, 112.45, 166.92, 152.45]",0,sum_Tarifs BForBank2024 V8_WEB,None
5,banque à distance,5,15.000000,"[31.64, 164.42, 562.91, 183.78]",0,sum_Tarifs BForBank2024 V8_WEB,None
6,espace client,6,8.000000,"[31.64, 184.86, 123.12, 205.65]",0,sum_Tarifs BForBank2024 V8_WEB,None
7,service client bforbank,7,8.000000,"[31.64, 184.86, 123.12, 205.65]",0,sum_Tarifs BForBank2024 V8_WEB,None
8,vos moyens et opérations de paiement,8,15.000000,"[31.64, 217.61, 562.91, 236.98]",0,sum_Tarifs BForBank2024 V8_WEB,None
9,virements,9,8.000000,"[31.64, 238.06, 144.79, 268.45]",0,sum_Tarifs BForBank2024 V8_WEB,None
